<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E5%88%A9%E7%94%A8qwen%E6%A8%A1%E5%9E%8B%E8%BF%9B%E8%A1%8C%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96_async%E6%8B%93%E5%B1%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 换另一本小说
- [ ] 制作一个foo的summary和一个foo的抽取函数
- [ ] 模拟抽取流程

先从drive上找到小说

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!ls /content/drive/MyDrive/Wuxia/input/

1819部精校小说.zip  classic_50.zip  parts_1.zip  parts_3.zip  parts_5.zip  parts_7.zip	parts_9.zip
all_direct	    parts_0.zip     parts_2.zip  parts_4.zip  parts_6.zip  parts_8.zip


In [1]:
part_id = 11

这个zip在

https://drive.google.com/file/d/101g9BfaKLNv10P1C1wuePTdbR6rwKRpb/view?usp=sharing

In [2]:
# part_id = 10

# input_file = f"/content/drive/MyDrive/Wuxia/input/parts_" + str(part_id) + ".zip"

input_file = "/content/drive/MyDrive/Wuxia/input/classic_50.zip"

import os
import zipfile
import shutil

output_dir = "/content/input"

# 解压input_file到output_dir
with zipfile.ZipFile(input_file,"r") as zip_ref:
    zip_ref.extractall(output_dir)

In [3]:


import os
import chardet

output_folder = "/content/input/output2"
# output_folder = f"/content/output/parts_{part_id}"


file_names = []

# 遍历所有txt文件
for root, dirs, files in os.walk(output_folder):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)

            with open(file_path, 'rb') as f:
                content = f.read(1000)
            result = chardet.detect(content)

            if result['confidence'] < 0.9:
                print(f"Warning: {file_path} encoding confidence {result['confidence']:.0%} lower than 90%")
                continue

            file_names.append({
                "file_name": file_path,
                "encoding": result['encoding'],
                "confidence": result['confidence']
            })

In [4]:
for file_name in file_names[:5]:
    print(file_name)

{'file_name': '/content/input/output2/《盗墓笔记》（精校全本）作者：南派三叔.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《北京遇上西雅图》（精校全本）作者：薛晓璐.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《冰与火之歌05魔龙的狂舞》（精校全本）作者：乔治·R·R·马丁.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《全职高手》（校对全本）作者：蝴蝶蓝.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《大染坊》（精校全本）作者：陈杰.txt', 'encoding': 'GB2312', 'confidence': 0.99}


In [5]:
import re

def get_file_name(fname):
    last_split = fname.split('/')[-1]

    pattern = r'《(.+)》'
    m = re.match(pattern, last_split)
    if m:
        # print("match")
        return m.group(1) + '.txt'
    else:
        return last_split

fname = "/content/output/output2/《凡人修仙传》(精校全本)作者:忘语.txt"

print(get_file_name(fname)[:-4])

凡人修仙传


In [6]:
def predict_chapater(line):
    line = line.strip()
    line = line.strip('-=*')
    if len(line) > 100:
        return -1
    if len(line) == 0:
        return -1
    is_short = len(line) < 10

    head_5_char = line[:min(5,len(line))]

    small_line = line.lower()
    line = line[:20]
    has_di = line.find('第', 0 ) >= 0
    di_in_head = head_5_char.find('第', 0) >= 0

    has_zhang = False
    zhang_in_head = False

    zhang_word = ['章', '集', '卷','回','期','节']



    for w in zhang_word:
        if head_5_char.find(w, 0) >= 0:
            zhang_in_head = True
            break

    for w in zhang_word:
        if line.find(w, 0) >= 0:
            has_zhang = True
            break

    has_digital = False
    digital_word = ['0','1','2','3','4','5','6','7','8','9',\
                    '一','二','三','四','五','六','七','八','九']

    only_digital = False
    digital_count = 0
    for ch in line:
        if ch in digital_word:
            digital_count += 1
    if digital_count == len(line):
        only_digital = True

    digital_in_head = False

    for w in digital_word:
        if head_5_char.find(w, 0) >= 0:
            digital_in_head = True
            break
    for w in digital_word:
        if line.find(w, 0) >= 0:
            has_digital = True
            break
    if has_di and has_zhang and has_digital and (di_in_head or digital_in_head or zhang_in_head):
        return 3
    if di_in_head and digital_in_head and is_short:
        return 2
    if zhang_in_head and digital_in_head and is_short:
        return 2
    # if only_digital and is_short:
    #     return 2
    return -1


In [8]:

!pip install -q transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install -q auto-gptq optimum

In [7]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract-csv", trust_remote_code=True)

num_workers = 2

models = []

for i in range(num_workers):
    models.append(None)
    models[i] = AutoModelForCausalLM.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract-csv", device_map="auto", trust_remote_code=True)

In [26]:
for model in models:
    model = model.eval()

In [8]:
print(len(tokenizer.tokenize("我是一只猫")))

4


In [9]:
print(models[0].generation_config)

GenerationConfig {
  "chat_format": "chatml",
  "do_sample": true,
  "eos_token_id": 151643,
  "max_new_tokens": 512,
  "max_window_size": 6144,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "top_k": 0,
  "top_p": 0.8
}



In [10]:
#@title  定义divide函数，用来切分超长文本
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, mid_len // 2, len(s)-mid_len // 2)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

In [11]:
import os

save_folder = "/content/output_in_chunk"

TOKEN_PER_TRUNK = 600

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

import codecs

from tqdm import tqdm
# for file_name_data in tqdm(file_names):
for file_name_data in file_names:
    file_name = file_name_data['file_name']
    if file_name.find("《大染坊》",0)<0:
        # 先找一本进行调试
        continue

    book_name = get_file_name(file_name)
    if len(book_name) > 4:
        book_name = book_name[:-4]
    fencoding = file_name_data['encoding']
    f = codecs.open(file_name, 'r', encoding=fencoding, errors = 'ignore')
    raw_text = f.read()
    lines = raw_text.splitlines()
    f.close()

    head_count = 0

    is_heads = [predict_chapater(line) > 0 for line in lines]

    token_per_line = []

    for line in tqdm(lines):
        if line.strip() == '':
            token_per_line.append(0)
            continue
        n_token = len(tokenizer.tokenize(line.strip()))
        token_per_line.append(n_token)

    # 这里还要处理 如果单个line超过TOKEN_PER_TRUNK, 需要调用strong_divide进行分裂
    # 同时第一个line保留原来的is_head, 其他的都是false


    for line, is_head in zip(lines, is_heads):
        if is_head:
            head_count += 1



    print(file_name_data)
    print(head_count)
    break

100%|██████████| 7492/7492 [00:00<00:00, 16838.47it/s]

{'file_name': '/content/input/output2/《大染坊》（精校全本）作者：陈杰.txt', 'encoding': 'GB2312', 'confidence': 0.99}
30


In [12]:
print(sum(token_per_line)/600)

475.4166666666667


In [13]:
chunks = []

current_n = 0
current_chunk = ""
current_chunk_is_head = True
last_chunk_is_head = False

for line, is_head, n_token in zip(lines, is_heads, token_per_line):
    # 如果is_head且之前不是head
    # 如果is_head且之前的current_n > TOKEN_PER_TRUNK // 2 需要结算
    # 如果n_token + current_n > TOKEN_PER_TRUNK 需要结算
    count_flag = False

    if is_head and not last_chunk_is_head:
        count_flag = True
    elif is_head and last_chunk_is_head and current_n > TOKEN_PER_TRUNK // 2:
        count_flag = True
    elif n_token + current_n > TOKEN_PER_TRUNK:
        count_flag = True

    if count_flag and current_chunk.strip() != "":
        chunk_data = {
            "text": current_chunk,
            "is_head": current_chunk_is_head
        }
        chunks.append(chunk_data)
        last_chunk_is_head = current_chunk_is_head
        current_chunk_is_head = is_head
        current_chunk = ""
        current_n = 0

    current_chunk += line + "\n"
    current_n += n_token

if current_chunk.strip() != "":
    chunk_data = {
        "text": current_chunk,
        "is_head": current_chunk_is_head
    }
    chunks.append(chunk_data)

In [14]:
print(sum([n_token>TOKEN_PER_TRUNK for n_token in token_per_line]))
print(sum([ chunk["is_head"] == True for chunk in chunks ]))
print(len(chunks))

0
31
524


In [15]:
import copy
# 做个备份
org_chunk = copy.deepcopy(chunks)

In [16]:
def foo_summary( text ):
    return text[:10] + "..." + text[-10:]

def foo_extract( text):
    return "dialogue text | said by | speaker\n" + text[:20]

In [17]:
summary_prompt = """请对以下小说文本进行总结，原文:"""
extract_prompt = """给定input paragraph，以及相应的summary，抽取每一句对话的内容，判断每一句话的说话人 以<dialogue text> | said by | <speaker>的形式输出成csv格式"""


In [30]:
# for data in chunks:
#     if "summary" in data:
#         del data["summary"]

plausible_start_ids = [ id for id, chunk in enumerate(chunks) if chunk["is_head"] ]


# 先解决所有的summary

if len(plausible_start_ids) < num_workers:
    step = len(chunks) // (num_workers*2)
    for i in range(1,(num_workers*2)):
        plausible_start_ids.append(i * step)

# 记录last summary

valid_ids = [(id,"") for id in plausible_start_ids]

count_deal = 0

import random

# 定义异步函数来处理输入
async def process_summary(model, input):
    current_summary, _ = model.chat(tokenizer, input, history=[])
    return current_summary

import asyncio
# 主函数来并行处理输入
async def process_summaires_in_async(datas):
    results = await asyncio.gather( \
        *(process_summary( models[id], data['input']) for id, data in enumerate(datas)))

    for data, result in zip(datas,results):
        data['output'] = result


from tqdm import tqdm
for _ in tqdm(range( len(chunks) )):
    if count_deal >= len(chunks):
        if count_deal != len(chunks):
            print('warning! invalid count_deal ', count_deal)
        break

    # 仍然valid的ids是 plausible_ids_tuple 中且对应的 tuple的第二个字段不为"FINISHED"的
    valid_ids = [ (id,last_summary) for id,last_summary in valid_ids if last_summary.strip() != "FINISHED" ]

    if len(valid_ids)>num_workers:
        job_ids = random.sample(range(len(valid_ids)), num_workers)
    else:
        job_ids = range(len(valid_ids))

    batch_inputs = []

    for job_id in job_ids:
        id = valid_ids[job_id][0]
        last_summary = valid_ids[job_id][1]

        #批量处理摘要
        text = chunks[id]["text"]
        if last_summary.strip() != "":
            text = last_summary + "\n" + text

        batch_inputs.append(summary_prompt + text)

    datas = [{"input":input} for input in batch_inputs]
    await process_summaires_in_async(datas)
    batch_outputs = [ data["output"] for data in datas]

    for job_id, current_summary in zip(job_ids,batch_outputs):
        id = valid_ids[job_id][0]

        chunks[id]["summary"] = current_summary
        chunks[id]["summary_order"] = count_deal
        count_deal += 1

        finished_flag = False

        if id + 1 >= len(chunks):
            finished_flag = True
        elif chunks[id + 1]["is_head"]:
            finished_flag = True
        elif "summary" in chunks[id + 1]:
            print("warning! , 冲撞 id = ",id)
            finished_flag = True

        if finished_flag:
            valid_ids[job_id] = (id,"FINISHED")
        else:
            valid_ids[job_id] = (id+1,current_summary)




 50%|█████     | 263/524 [57:44<57:18, 13.17s/it]


我们来实验双模型并行的方法

In [ ]:
n = len(chunks)

# 定义异步函数来处理输入
async def process_extract(model, input_text):
    csv_response, _ = model.chat(tokenizer, input_text, system = extract_prompt, history=[])
    return csv_response

import asyncio
# 主函数来并行处理输入
async def process_extract_in_async(datas):
    results = await asyncio.gather( \
        *(process_extract( models[id], data['input']) for id, data in enumerate(datas)))

    for data, result in zip(datas,results):
        data['output'] = result



for id in tqdm(range(0,n,2)):
    right_id = min(n, id + 2)
    datas = []
    for chunk in chunks[id:right_id]:
        text = chunk["text"]
        summary = chunk["summary"]
        input_text = f"paragraph:\n{text}\n\nsummary:\n{summary}"
        datas.append({"input":input_text})

    await process_extract_in_async(datas)

    for sub_id, data in enumerate(datas):
        chunks[id+sub_id]["csv_response"] = data["output"]

  2%|▏         | 6/262 [01:53<1:32:28, 21.67s/it]

In [ ]:
final_save_name = f"/content/drive/MyDrive/CardBuild/exp0130/daranfang.txt"

with open(final_save_name, "w", encoding="utf-8") as f:
    for chunk in chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + "\n")